In [98]:
import requests
import pandas as pd

states = []
latitude = []
longitude = []

url = 'https://www.fema.gov/api/open/v2/FemaRegions/8a8064df-f1f7-4f0c-a3ae-b31fce82778b'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    states.extend(data["FemaRegions"][0]["states"])

    # Iterate over each set of coordinates in the nested lists
    for region in data["FemaRegions"][0]["regionGeometry"]["coordinates"]:
        for polygon in region:
            for coord in polygon:
                longitude.append(coord[0])
                latitude.append(coord[1])

# Repeat states to match the length of latitude and longitude
repeated_states = states * (len(latitude) // len(states)) + states[:len(latitude) % len(states)]

# Create the DataFrame
df = pd.DataFrame({
    'State': repeated_states,
    'Latitude': latitude,
    'Longitude': longitude
})

print(df)


     State   Latitude  Longitude
0       NM  26.078077 -97.172048
1       TX  26.072277 -97.179586
2       OK  26.250894 -97.209167
3       LA  26.348901 -97.226563
4       AR  26.419652 -97.251504
...    ...        ...        ...
1102    OK  36.998114 -95.785749
1103    LA  37.001019 -95.526019
1104    AR  37.000616 -95.406622
1105    NM  37.001478 -95.071931
1106    TX  37.000779 -95.032745

[1107 rows x 3 columns]


In [99]:
import requests
import pandas as pd

url = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$top=1107"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

    # Initialize separate lists for each field
    states = []
    incident_types = []
    incident_begin_dates = []
    incident_end_dates = []

    # Iterate over the data and append to the respective lists
    for item in data["DisasterDeclarationsSummaries"]:
        states.append(item['state'])
        incident_types.append(item['incidentType'])
        incident_begin_dates.append(item['incidentBeginDate'])
        incident_end_dates.append(item['incidentEndDate'])

    # Create a DataFrame
    disaster_df = pd.DataFrame({
        'State': states,
        'Incident Type': incident_types,
        'Incident Begin Date': incident_begin_dates,
        'Incident End Date': incident_end_dates
    })

    print(disaster_df.shape)

(1107, 4)


In [100]:
disaster_df.head()

,State,Incident Type,Incident Begin Date,Incident End Date
0,NH,Hurricane,2011-08-26T00:00:00.000Z,2011-09-06T00:00:00.000Z
1,RI,Fire,2023-04-14T00:00:00.000Z,2023-04-16T00:00:00.000Z
2,KS,Fire,2023-04-13T00:00:00.000Z,2023-04-16T00:00:00.000Z
3,NH,Hurricane,2011-08-26T00:00:00.000Z,2011-09-06T00:00:00.000Z
4,CO,Flood,2023-06-08T00:00:00.000Z,2023-06-23T00:00:00.000Z


In [101]:
final_df = pd.merge(df, disaster_df, on='State')

In [102]:
final_df.head()

,State,Latitude,Longitude,Incident Type,Incident Begin Date,Incident End Date
0,NM,26.078077,-97.172048,Fire,2021-04-26T00:00:00.000Z,None
1,NM,26.078077,-97.172048,Fire,2019-03-07T00:00:00.000Z,2019-03-13T00:00:00.000Z
2,NM,26.078077,-97.172048,Fire,2018-06-07T00:00:00.000Z,2019-06-19T00:00:00.000Z
3,NM,26.078077,-97.172048,Fire,2016-07-13T00:00:00.000Z,2016-08-18T00:00:00.000Z
4,NM,26.078077,-97.172048,Fire,2016-06-14T00:00:00.000Z,2016-08-10T00:00:00.000Z


In [103]:
final_df.shape

(16839, 6)